# State Construction Task

In [51]:
from datetime import datetime
import itertools
import io

import numpy as np
import scipy.sparse as sp
import tensorflow as tf

from tqdm import tqdm
import matplotlib.pyplot as plt

generates equidistributed points on unit nd-sphere (see [here](https://www.cmu.edu/biolphys/deserno/pdf/sphere_equi.pdf))

In [52]:
def equidistributed_points(dim, dist):
    with np.errstate(divide='ignore'):
        if np.isnan(dist):
            dist = 4 * np.pi
        if dim == 2:
            phis = np.arange(0, 2 * np.pi, dist)
            return np.vstack([np.cos(phis), np.sin(phis)])
        else:
            slices = []
            for phi in np.arange(0, np.pi, dist):
                proj_points = equidistributed_points(dim - 1, dist / np.sin(phi))
                points = np.vstack([np.full((1, proj_points.shape[1]), np.cos(phi)), np.sin(phi) * proj_points])
                slices.append(points)
            return np.hstack(slices)

## solve using Tensorflow

parametric hamiltonian generator with `tf.Variable`

In [53]:
dim = 2

def h_generator(batch_size):
    def generate_parametric(n, dv_lambda, name):
        params = tf.get_variable(name, dtype='float64', shape=(batch_size, n), initializer=tf.truncated_normal_initializer)
        params = tf.cast(params, 'complex128')
        places = tf.constant(np.stack([dv_lambda(c) for c in range(n)]), dtype='complex128')
        return tf.tensordot(params, places, axes=[1, 0])
    

    neardiag = lambda i : sp.coo_matrix(
        ([1, 1], ([i, (i + 1) % dim], [(i + 1) % dim, i % dim])), shape=(dim, dim)).toarray()

    diag = lambda i : sp.coo_matrix(([1], ([i], [i])), shape=(dim, dim)).toarray()
    h = generate_parametric(dim - 1, neardiag, 'neardiag') + generate_parametric(dim, diag, 'diag')
    
    return h

design own solver

In [54]:
def batch_solve_tensorflow(dim, h_generator, source, targets, batch_size):
    tf.reset_default_graph()
    
    with tf.device('/cpu:0'):
        source = tf.tile(tf.expand_dims(tf.constant(source, dtype='complex128'), axis=0), [batch_size, 1])
        target = tf.placeholder(dtype='complex128', shape=(batch_size, dim))

        h = h_generator(batch_size)

        gate = tf.linalg.expm(1j * h)
        estimated_target = tf.matmul(gate, tf.expand_dims(source, 2))

        target_dot = tf.matmul(tf.expand_dims(target, 2), estimated_target, adjoint_a=True)
        fidelities = tf.abs(target_dot)
        loss = 1 - tf.reduce_mean(fidelities)

        optimize_op = tf.train.MomentumOptimizer(momentum=0.8, learning_rate=0.5).minimize(loss)

    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
        for t in tqdm(targets):
            init_op = tf.global_variables_initializer()
            session.run(init_op)
            for i in range(5000): # todo, improve stop
                session.run(optimize_op, feed_dict={target: t})

            yield session.run(fidelities, feed_dict={target: t})

In [83]:
angle = 0.2
replica = 32
batch_size = 1024

source = np.array([1] + [0] * (dim - 1))
map_points = equidistributed_points(2 * dim, angle).T
targets = map_points[:, :dim] + 1j * map_points[:, dim:]
targets_count = targets.shape[0]
print('count: ', targets_count * replica)

ts = np.repeat(targets, replica, axis=0)
if (targets_count * replica) % batch_size:
    ts = np.vstack([ts, np.zeros((batch_size - ((targets_count * replica) % batch_size), dim))])
ts = np.split(ts, ts.shape[0] // batch_size)

res = list(batch_solve_tensorflow(dim, h_generator, source, ts, batch_size))

res = np.vstack(res)
res = np.squeeze(res)
res = res[:targets_count * replica]
res = res.reshape((targets_count, replica))
fidelities = np.max(res, axis=1)

count:  81920











  0%|          | 0/80 [00:00<?, ?it/s]








  1%|▏         | 1/80 [00:16<21:29, 16.32s/it]








  2%|▎         | 2/80 [00:32<21:10, 16.29s/it]








  4%|▍         | 3/80 [00:48<20:51, 16.25s/it]








  5%|▌         | 4/80 [01:04<20:31, 16.20s/it]








  6%|▋         | 5/80 [01:20<20:13, 16.17s/it]








  8%|▊         | 6/80 [01:36<19:55, 16.15s/it]








  9%|▉         | 7/80 [01:53<19:38, 16.14s/it]








 10%|█         | 8/80 [02:09<19:20, 16.12s/it]








 11%|█▏        | 9/80 [02:25<19:04, 16.12s/it]








 12%|█▎        | 10/80 [02:41<18:47, 16.11s/it]








 14%|█▍        | 11/80 [02:57<18:31, 16.11s/it]








 15%|█▌        | 12/80 [03:13<18:15, 16.10s/it]








 16%|█▋        | 13/80 [03:29<17:58, 16.10s/it]








 18%|█▊        | 14/80 [03:45<17:42, 16.10s/it]








 19%|█▉        | 15/80 [04:01<17:26, 16.11s/it]








 20%|██        | 16/80 [04:18<17:11, 16.12s/it]








 21%|██▏       | 17/80 [04:34<16:55, 16.11s/it]








 

In [84]:
f_min = fidelities.min()
d = np.cos(angle)
MGF = (np.sqrt(d * f_min) - np.sqrt((1 - d) * (1 - f_min))) ** 2

print('d: ', d)
print('minimum fidelity: ', f_min)
print('average fidelity: ', fidelities.mean())
print('MGF: ', MGF)

d:  0.9800665778412416
minimum fidelity:  0.9962342658104231
average fidelity:  0.9999549162417709
MGF:  0.9593289797444705


In [74]:
worst = targets[fidelities.argmin(),:]
worst = np.repeat(np.expand_dims(worst, axis=0), 1024, axis=0)
worst_fidelity = list(batch_solve_tensorflow(dim, h_generator, source, [worst], 1024))[0]






  0%|          | 0/1 [00:00<?, ?it/s]




100%|██████████| 1/1 [00:16<00:00, 16.42s/it]